In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import xlrd
from py2neo import Graph,Node,Relationship
from py2neo.database import ClientError
import re

In [2]:
# option_entities_file = 'entities.csv'
# option_relations_file = 'relations.txt'
# option_properties_file = 'properties.txt'
option_deleteAllFlag = 0
option_demoFlag = 1
option_read_csv_row =  -1   # -1 for all rows, positive number k for first k rows.
option_userName = 'neo4j'
option_passWord = '321321'
option_config_file = r''
option_config_dir = r''
option_entities_dir = ''
option_column_prefix = "" 
option_label = ""
option_neo4j_ip = "localhost"
option_neo4j_port = "7474"

option_pas = True
option_intesrv = True
option_decoration = False
option_decoration_S = False
option_parts = False
option_data0909 = True
option_remote_ty = False
option_remote_thu = False
option_remote_ty_64 = False
option_remote_ty_22 = False
option_remote = False
# option_remote = option_remote_ty or option_remote_thu or option_remote_ty_64

if option_pas == True:
    option_config_dir = r'./config'
#     option_config_dir = r'E:/天远/知识图谱/天远知识图谱文档/数据血缘/PAS/'
    option_config_file = r'%s/%s'%(option_config_dir, "PAS_config.xlsx")
    option_entities_dir = './data/pas/'
    option_column_prefix = "" 
    option_label = "数据起源"

if option_intesrv == True:
    option_config_dir = r'E:/天远/知识图谱/天远知识图谱文档/天远百度/config/config_intesrv'
    if option_remote == True:
        option_config_dir = r'./data/config'
    option_config_file = r'%s/%s'%(option_config_dir, "config_v2.3.1.xlsx")
    option_entities_dir = './data/intesrv_real_2.0/'
    option_column_prefix = "plt_" 
#     option_label = "智能服务"
    option_label = "维修服务"
    
if option_decoration == True:
    option_config_dir = r'E:/天远/知识图谱/天远知识图谱文档/天远百度/config/config_manual'
    if option_remote == True:
        option_config_dir = './data/config'
    option_config_file = r'%s/%s'%(option_config_dir, "config_manual_v1.3.3.xlsx")
    option_entities_dir = './data/decoration/'
    option_column_prefix = "" 
#     option_label = "装修手册"
    option_label = "故障诊断"
    
if option_decoration_S == True:
    option_config_dir = r'E:/天远/知识图谱/天远知识图谱文档/天远百度/config/config_manual'
    if option_remote == True:
        option_config_dir = './data/config'
    option_config_file = r'%s/%s'%(option_config_dir, "config_manual_S_v1.3.2.xlsx")

    option_entities_dir = './data/decoration/'
    option_column_prefix = "" 
#     option_label = "装修手册"
    option_label = "故障诊断"
    
if option_parts == True:
    option_config_dir = r'E:/天远/知识图谱/天远知识图谱文档/天远百度/config/config_product_structure'
    if option_remote == True:
        option_config_dir = './data/config'
    option_config_file = r'%s/%s'%(option_config_dir, "config_product_structure_v1.2.xlsx")
    option_entities_dir = './data/part/'
    option_column_prefix = "" 
    option_label = "产品结构"
    
if option_data0909 == True:
    option_config_dir = r'E:/天远/知识图谱/天远知识图谱文档/天远百度/config/config_intesrv'
    option_entities_dir = r'E:/天远/知识图谱/天远知识图谱文档/天远百度/newdata0909/'
    if option_remote == True:
        option_config_dir = './data/config'
        option_entities_dir = './data/data0909/'
    option_config_file = r'%s/%s'%(option_config_dir, "config_data0909_v1.4.xlsx")
    option_column_prefix = "" 
    option_label = "维修服务"
    
if option_remote_ty == True:
    option_passWord = '321321'
    option_neo4j_ip = "101.6.15.221"
    
if option_remote_thu == True:
    option_passWord = '321321'
    option_neo4j_ip = "101.6.240.89"

if option_remote_ty_64 == True:
    option_passWord = '321321'
    option_neo4j_ip = "192.168.30.64"
    
if option_remote_ty_22 == True:
    option_passWord = '321321'
    option_neo4j_ip = "192.168.111.22"

## 读取配置表

In [3]:
class Config:
    MAIN_FOREIGN_SHEET_NAME = '主外键对应表'
    CONCEPT_SHEET_NAME = '模式图的概念与数据库对应表'
    RELATION_SHEET_NAME = '关系表'

    DICT_CONCEPT = {"模式图中概念":"concept_ch",
                   "显示名":"display_name",
                   "数据库表/属性":"table_and_column",
                   "类型":"type",
                   "逻辑代码连接":"code",
                   "内部属性":"inner_properties_ch",
                   "内部属性对应数据库属性":"properties_col_str",
                   "逻辑代码所需数据库表":"compound_table_str",
                   "所属域":"domain"}
    DICT_CONCEPT_REVERSE = {value:key for key,value in DICT_CONCEPT.items()}
    DICT_RELATION = {"关系主语":"sub",
                   "关系谓语":"predicate",
                   "关系宾语":"obj",
                   "关系类型":"type",
                   "关系所在数据库表":"table",
                   "关系主语对应数据库属性":"sub_col",
                   "关系宾语对应数据库属性":"obj_col",
                    "关系属性":"prop_ch_str",
                    "关系属性对应数据库属性":"prop_col_str"}
    DICT_RELATION_REVERSE = {value:key for key,value in DICT_RELATION.items()}
    def __init__(self, file_path, debug=False):
        self.__initVariable(file_path, debug)
    
    def __initVariable(self, file_path, debug):
        self.__file_path = file_path
        self.__mainForeign = None
        self.__concept = None
        self.__relation = None
        self.__tables_name = []
        self.__debug = debug

    def readAllContent(self):
        self.__readConcept()
        self.__readRelation()
        self.__readMainForeign()
        self.__getAllTablesInTopoSort()    
        
    def __split_compound_table(paragraph):
        results = []
        for sentence in re.split(",|，", paragraph):        
            compound = {}
            compound['main'] = sentence.split('-')[0]
            compound['foreign'] = sentence.split('-')[1]
            compound['main_table'] = compound['main'].split('.')[0]
            compound['main_col'] = option_column_prefix + compound['main'].split('.')[1]
            compound['foreign_table'] = compound['foreign'].split('.')[0]
            compound['foreign_col'] = option_column_prefix + compound['foreign'].split('.')[1]
            results.append(compound)
        return results   
    
    def __readConcept(self):
        df = pd.read_excel(open(self.__file_path, 'rb'),sheet_name=self.CONCEPT_SHEET_NAME)
        for col in df.columns:
            df[col] = df[col].astype(str).str.lower()
            df[col] = df[col].str.replace("\n","")
        df = df.rename(index=str, columns=self.DICT_CONCEPT)
        # print(df)
        df['table'] = None
        df['col'] = None
        # df['test'] = df['concept_ch'].apply(split_compound_table)
        # print(df)
        # df[df['type'] == '直接']['table'] = df[df['type'] == '直接']['table_and_column'].astype(str).apply(lambda x: x.split('.')[0])
        df.loc[df['type'] == '直接','table'] = df[df['type'] == '直接']['table_and_column'].astype(str).apply(lambda x: x.split('.')[0])
        df.loc[df['type'] == '直接','col'] = option_column_prefix + df[df['type'] == '直接']['table_and_column'].astype(str).apply(lambda x: x.split('.')[1])
        df.loc[df['type'] == '直接','properties_col'] = df[df['type'] == '直接']['properties_col_str'].astype(str).apply(lambda x: [option_column_prefix+y for y in re.split(",|，", x) if x != "nan"] )
        df.loc[df['type'] == '直接','properties_col_ch'] = df[df['type'] == '直接']['inner_properties_ch'].astype(str).apply(lambda x: [y for y in re.split(",|，", x) if x != "nan"] )
        df.loc[(df['type'] == '合成概念'), 'table'] = df[(df['type'] == '合成概念')]['table_and_column'].astype(str)
        df.loc[(df['type'] == '合成概念') & (df['compound_table_str'] != 'nan'), 'compound_table'] = df[(df['type'] == '合成概念') & (df['compound_table_str'] != 'nan')]['compound_table_str'].astype(str).apply(Config.__split_compound_table)
        self.__concept = df
        
        
    
    def __readRelation(self):
        df = pd.read_excel(open(self.__file_path, 'rb'),sheet_name=self.RELATION_SHEET_NAME)
        for col in df.columns:
            df[col] = df[col].astype(str).str.lower()
            df[col] = df[col].str.replace("\n","")
        df = df.rename(index=str, columns=self.DICT_RELATION)
        df['prop_ch'] = df['prop_ch_str'].astype(str).apply(lambda x: [y for y in re.split(",|，", x) if x != "nan"] )
        df['prop_col'] = df['prop_col_str'].astype(str).apply(lambda x: [option_column_prefix+y for y in re.split(",|，", x) if x != "nan"] )
        self.__relation = df
        
    def __readMainForeign(self):
        df = pd.read_excel(open(self.__file_path, 'rb'),sheet_name=self.MAIN_FOREIGN_SHEET_NAME)
        for col in df.columns:
            df[col] = df[col].astype(str).str.lower()
        df['primary_table'] = df['主键'].apply(lambda x: x.split('.')[0])
        df['primary_key'] = option_column_prefix + df['主键'].apply(lambda x: x.split('.')[1])
        df['foreign_table'] = df['外键'].apply(lambda x: x.split('.')[0])
        df['foreign_key'] = option_column_prefix + df['外键'].apply(lambda x: x.split('.')[1])
        if self.__debug:
            print(df)
        self.__mainForeign = df
        
    def __getAllTablesInTopoSort(self):
        self.__toposortTablesFromMainForeign()
        self.__addTablesFromConcept()
    
    def __toposortTablesFromMainForeign(self):
        tables_name = []
        df_foreign_tobedeleted = self.__mainForeign.copy()
        count = 0
        while len(df_foreign_tobedeleted) > 0:
            if self.__debug:
                print(count)
            primary_unique = df_foreign_tobedeleted['primary_table'].unique()
            foreign_uninue = df_foreign_tobedeleted['foreign_table'].unique()
            all_table = np.concatenate((primary_unique,foreign_uninue))
            tables_name_unordered = set(all_table)
            if self.__debug:
                print(tables_name_unordered)
            foreign_table = set(df_foreign_tobedeleted['foreign_table'])
            table = tables_name_unordered.difference(foreign_table).pop()
        #     print(tables_name_unordered.difference(foreign_table))
            tables_name.append(table)
        #     print(table)
            index_tobedeleted = df_foreign_tobedeleted.loc[df_foreign_tobedeleted['primary_table'] == table].index
        #     print(index_tobedeleted)
            df_foreign_tobedeleted.drop(index_tobedeleted, inplace=True)
        #     print(df_foreign_tobedeleted[['primary_table','foreign_table']])
            count += 1   
        if self.__debug:
            print(tables_name)

        # 只依赖其他表而没有被其他表依赖的表，并没有在上一步骤中被添加进来，所以在此处添加他们
        df_foreign_tobedeleted = self.__mainForeign.copy()
        primary_unique = df_foreign_tobedeleted['primary_table'].unique()
        foreign_uninue = df_foreign_tobedeleted['foreign_table'].unique()
        all_table = np.concatenate((primary_unique,foreign_uninue))
        tables_name_unordered = set(all_table)
        table_not_added = tables_name_unordered.difference(set(tables_name))
        while len(table_not_added) > 0:
            tables_name.append(table_not_added.pop())
        if self.__debug:
            print(tables_name)
        self.__tables_name += tables_name
    
    def __addTablesFromConcept(self):
        tables_name_from_concept = set(self.__concept['table'].dropna().unique())
        tables_name_now = set(self.__tables_name)
        tables_name_to_add = tables_name_from_concept.difference(tables_name_now)
        tables_name_to_add = list(tables_name_to_add)
        self.__tables_name += tables_name_to_add
        if self.__debug:
            print("debug:self.__tables_name=",self.__tables_name)
    
    def getConcept(self):
        return self.__concept
    
    def getRelation(self):
        return self.__relation
    
    def getMainForeign(self):
        return self.__mainForeign
    
    def getTables(self):
        return self.__tables_name
    
    def getConceptOfOneTable(self, table_name):
        concept = self.getConcept()
        return concept[concept['table'] == table_name].reset_index()
    
    def getRelationOfOneTable(self, table_name):
        relation = self.getRelation()
        return relation[relation['table'] == table_name].reset_index()

In [4]:
config = Config(option_config_file, debug=False)
config.readAllContent()

### 主外键对应表

In [5]:
df_foreign = config.getMainForeign()
df_foreign

,主键,外键,primary_table,primary_key,foreign_table,foreign_key
0,srvpersontrainrecord_falutpart.part,srvpersontrainrecord.part,srvpersontrainrecord_falutpart,part,srvpersontrainrecord,part
1,devicemodel_entity.devicemodelclass,srvpersontrainrecord.devicemodel,devicemodel_entity,devicemodelclass,srvpersontrainrecord,devicemodel
2,devicemodel_entity.devicemodelclass,devicemodel_relation.machine,devicemodel_entity,devicemodelclass,devicemodel_relation,machine
3,devicemodel_entity.devicemodelclass,devicemodel_relation.machineclass,devicemodel_entity,devicemodelclass,devicemodel_relation,machineclass
4,devicemodel_entity.devicemodelclass,devicemodel_relation.modelseries,devicemodel_entity,devicemodelclass,devicemodel_relation,modelseries
5,devicemodel_entity.devicemodelclass,devicemodel_relation.model,devicemodel_entity,devicemodelclass,devicemodel_relation,model
6,srvpersontrainrecord_falutpart.part,srvpersontrainrecord_falutpart_relation.parent...,srvpersontrainrecord_falutpart,part,srvpersontrainrecord_falutpart_relation,parentpart
7,srvpersontrainrecord_falutpart.part,srvpersontrainrecord_falutpart_relation.part,srvpersontrainrecord_falutpart,part,srvpersontrainrecord_falutpart_relation,part
8,srvpersonlevel.srvpersonname,srvpersoncar.srvpersonname,srvpersonlevel,srvpersonname,srvpersoncar,srvpersonname
9,srvpersonlevel.srvpersonname,stationarypoint.srvpersonname,srvpersonlevel,srvpersonname,stationarypoint,srvpersonname


#### 根据主外键关系，对表进行拓扑排序

In [6]:

tables_name = config.getTables()
tables_name

['devicemodel_entity',
 'srvpersontrainrecord_falutpart',
 'srvpersonlevel',
 'stationarypoint',
 'srvpersontrainrecord_falutpart_relation',
 'srvpersontrainrecord',
 'devicemodel_relation',
 'srvpersoncar']

### 关系表

In [7]:
df_relation = config.getRelation()
df_relation

,id,sub,predicate,obj,table,type,prop_ch_str,prop_col_str,sub_col,obj_col,prop_ch,prop_col
0,31,现场服务人员,隶属于,服务区,stationarypoint,nan,nan,nan,nan,nan,[],[]
1,32,现场服务人员,驻扎在,驻点,stationarypoint,nan,nan,nan,nan,nan,[],[]
2,33,现场服务人员,使用,服务人员使用车,srvpersoncar,nan,nan,nan,nan,nan,[],[]
3,34,驻点,管辖,服务区,stationarypoint,nan,nan,nan,nan,nan,[],[]
4,35,现场服务人员,培训过,机型分类,srvpersontrainrecord,nan,nan,nan,nan,nan,[],[]
5,36,现场服务人员,培训过,部件,srvpersontrainrecord,nan,nan,nan,nan,nan,[],[]
6,37,机型分类,包含,机型分类,devicemodel_relation,nan,nan,nan,machine,machineclass,[],[]
7,38,机型分类,包含,机型分类,devicemodel_relation,nan,nan,nan,machineclass,modelseries,[],[]
8,39,机型分类,包含,机型分类,devicemodel_relation,nan,nan,nan,machineclass,model,[],[]
9,40,机型分类,包含,机型分类,devicemodel_relation,nan,nan,nan,modelseries,model,[],[]


### 概念表

In [8]:
df_concept = config.getConcept()
df_concept

,concept_ch,display_name,table_and_column,type,inner_properties_ch,properties_col_str,code,compound_table_str,domain,table,col,properties_col,properties_col_ch,compound_table
0,现场服务人员,姓名,srvpersoncar.srvpersonname,直接,"姓名,级别",srvpersonname，srvpersonlevel,nan,nan,维修服务,srvpersoncar,srvpersonname,"[srvpersonname, srvpersonlevel]","[姓名, 级别]",NaN
1,服务人员使用车,车牌,srvpersoncar.srvcarid,直接,车号，车牌,"srvcarid,srvcarlicense",nan,nan,nan,srvpersoncar,srvcarid,"[srvcarid, srvcarlicense]","[车号, 车牌]",NaN
2,现场服务人员,姓名,srvpersonlevel.srvpersonname,直接,"姓名,级别",srvpersonname，srvpersonlevel,nan,nan,nan,srvpersonlevel,srvpersonname,"[srvpersonname, srvpersonlevel]","[姓名, 级别]",NaN
3,现场服务人员,姓名,stationarypoint.srvpersonname,直接,"姓名,级别",srvpersonname，srvpersonlevel,nan,nan,nan,stationarypoint,srvpersonname,"[srvpersonname, srvpersonlevel]","[姓名, 级别]",NaN
4,服务区,unique_id,stationarypoint.srvarea,直接,nan,nan,nan,nan,维修服务,stationarypoint,srvarea,[],[],NaN
5,驻点,名称,stationarypoint.stationarypointname,直接,"名称,经纬度","stationarypointname,stationarypointcoordinate",nan,nan,nan,stationarypoint,stationarypointname,"[stationarypointname, stationarypointcoordinate]","[名称, 经纬度]",NaN
6,现场服务人员,姓名,srvpersontrainrecord.srvpersonname,直接,"姓名,级别",srvpersonname，srvpersonlevel,nan,nan,nan,srvpersontrainrecord,srvpersonname,"[srvpersonname, srvpersonlevel]","[姓名, 级别]",NaN
7,部件,unique_id,srvpersontrainrecord_falutpart.part,直接,nan,nan,nan,nan,nan,srvpersontrainrecord_falutpart,part,[],[],NaN
8,机型分类,unique_id,devicemodel_entity.devicemodelclass,直接,层级,class,nan,nan,nan,devicemodel_entity,devicemodelclass,[class],[层级],NaN


## 读取实体数据

In [9]:
# table_name = 'srvpersontrainrecord'
# path = option_entities_dir + table_name + filetype
# df = pd.read_excel(path)
# for key in list(df.columns.values):  # 如果不这样的话，可能会读入int64的数字，但是neo4j不支持int64
# #     df[key] = df[key].astype("str")
#     df[key] = df[key].astype("str").apply(lambda s: s.strip())

# df.loc[0,'srvpersonlevel'] != 'nan'

In [10]:
# table = tables['plt_intesrv_tyjob']
# print(table.loc[1445,'plt_jobsummary'])
# table[1440:1450]

In [11]:
# filetype = ".xlsx"
# filetype = ".csv"

tables = {}
total_length = 0
for table_name in tables_name:
#     print("start to access table " + table_name)
    try:
        filetype = ".csv"
        path = option_entities_dir + table_name + filetype
        df = pd.read_csv(path,sep=';',encoding='utf-16')
    except FileNotFoundError:
        filetype = ".xlsx"
        path = option_entities_dir + table_name + filetype
        df = pd.read_excel(path)
    for key in list(df.columns.values):  # 如果不这样的话，可能会读入int64的数字，但是neo4j不支持int64
#         df[key] = df[key].astype("object")
        df[key] = df[key].astype("str").apply(lambda s: s.strip())
    print(table_name, df.shape[0], df.shape[1]) 
    total_length += df.shape[0]
    row_use = df.shape[0] if option_read_csv_row == -1 else option_read_csv_row
    tables[table_name] = df[:row_use]
#     print("end to access table " + table_name)
# tables
total_length # 60460

devicemodel_entity 70 2
srvpersontrainrecord_falutpart 12 2
srvpersonlevel 57 5
stationarypoint 54 5
srvpersontrainrecord_falutpart_relation 12 3
srvpersontrainrecord 176 7
devicemodel_relation 78 4
srvpersoncar 82 7


541

## 导入Neo4j

In [12]:
graph = Graph('http://%s:%s'%(option_neo4j_ip,option_neo4j_port),username=option_userName,password=option_passWord)

In [13]:
graph.create(Node("TEST",value="yaoyao"))

In [14]:
graph.run("MATCH (n:%s) OPTIONAL MATCH (n)-[r]-() DELETE r,n"%("TEST")) 

In [15]:
if option_deleteAllFlag == 1:
    # 只删除本个label域下的节点和关系
    graph.run("MATCH (n:%s) OPTIONAL MATCH (n)-[r]-() DELETE r,n"%(option_label)) 
#     graph.delete_all()


In [16]:
if option_deleteAllFlag == 1:
    option_label_tmp = option_label
    option_label = "故障诊断"
    print(option_label)
    # 只删除本个label域下的节点和关系
    graph.run("MATCH (n:%s) OPTIONAL MATCH (n)-[r]-() DELETE r,n"%(option_label)) 
    option_label = option_label_tmp

In [17]:
if option_deleteAllFlag == 1:
    # 删除所有的关系
    graph.run("start r=relationship(*) delete r;")

In [18]:
if option_deleteAllFlag == 1:
    # 删除所有的节点和关系
    graph.delete_all()

### 模式图

In [19]:
dict_concept_reverse = Config.DICT_CONCEPT_REVERSE
dict_relation_reverse = Config.DICT_RELATION_REVERSE

In [20]:
df_concept.head()

,concept_ch,display_name,table_and_column,type,inner_properties_ch,properties_col_str,code,compound_table_str,domain,table,col,properties_col,properties_col_ch,compound_table
0,现场服务人员,姓名,srvpersoncar.srvpersonname,直接,"姓名,级别",srvpersonname，srvpersonlevel,nan,nan,维修服务,srvpersoncar,srvpersonname,"[srvpersonname, srvpersonlevel]","[姓名, 级别]",NaN
1,服务人员使用车,车牌,srvpersoncar.srvcarid,直接,车号，车牌,"srvcarid,srvcarlicense",nan,nan,nan,srvpersoncar,srvcarid,"[srvcarid, srvcarlicense]","[车号, 车牌]",NaN
2,现场服务人员,姓名,srvpersonlevel.srvpersonname,直接,"姓名,级别",srvpersonname，srvpersonlevel,nan,nan,nan,srvpersonlevel,srvpersonname,"[srvpersonname, srvpersonlevel]","[姓名, 级别]",NaN
3,现场服务人员,姓名,stationarypoint.srvpersonname,直接,"姓名,级别",srvpersonname，srvpersonlevel,nan,nan,nan,stationarypoint,srvpersonname,"[srvpersonname, srvpersonlevel]","[姓名, 级别]",NaN
4,服务区,unique_id,stationarypoint.srvarea,直接,nan,nan,nan,nan,维修服务,stationarypoint,srvarea,[],[],NaN


In [21]:
nodes_concept = {}
for index in tqdm(list(df_concept.index)):
    index = int(index)
#     print(index)
#     print(df_concept.iloc[index]['concept_ch'])
    key = df_concept.iloc[index]['concept_ch']
    nodeBefore = graph.nodes.match("概念节点",key).first()
    if nodeBefore is None:
        nodes_concept[key] = Node(key)
        nodes_concept[key].add_label("概念节点")
        if option_label != "":
            nodes_concept[key].add_label(option_label)
            nodes_concept[key]['domain_name'] = option_label
            nodes_concept[key]['display_name'] = key
            nodes_concept[key]['type'] = key
            nodes_concept[key]['value'] = key

        # 为概念节点增加属性，把该有的东西都放进去
        for col in list(df_concept.columns.values):
            col_chinese = dict_concept_reverse[col] if col in dict_concept_reverse.keys() else col
            nodes_concept[key][col_chinese] = str(df_concept[col][index]) if df_concept[col][index] == df_concept[col][index] else ""
    #     print(nodes_concept[key])
        graph.create(nodes_concept[key])
    else:
        nodes_concept[key] = nodeBefore

100%|███████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 346.92it/s]


In [22]:
df_relation.head()

,id,sub,predicate,obj,table,type,prop_ch_str,prop_col_str,sub_col,obj_col,prop_ch,prop_col
0,31,现场服务人员,隶属于,服务区,stationarypoint,nan,nan,nan,nan,nan,[],[]
1,32,现场服务人员,驻扎在,驻点,stationarypoint,nan,nan,nan,nan,nan,[],[]
2,33,现场服务人员,使用,服务人员使用车,srvpersoncar,nan,nan,nan,nan,nan,[],[]
3,34,驻点,管辖,服务区,stationarypoint,nan,nan,nan,nan,nan,[],[]
4,35,现场服务人员,培训过,机型分类,srvpersontrainrecord,nan,nan,nan,nan,nan,[],[]


In [23]:
for index in tqdm(list(df_relation.index)):
    index = int(index)
#     print(index)
    sub = df_relation.iloc[index]['sub']
    obj = df_relation.iloc[index]['obj']
#     try:
#     print(sub)
#     print(obj)
    aRelation = Relationship(nodes_concept[sub], df_relation.iloc[index]['predicate'], nodes_concept[obj] )
    aRelation["id_relation"] = df_relation.iloc[index]['id']
    for col in list(df_relation.columns.values):
        col_chinese = dict_relation_reverse[col] if col in dict_relation_reverse.keys() else col
        aRelation[col_chinese] = df_relation[col][index] if df_relation[col][index] == df_relation[col][index] else ""
     
    graph.create(aRelation)
#     except(KeyError):
#         pass

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 24.61it/s]


### 合成概念函数test

In [24]:
def test1(df):
#     print(df)
    return df['plt_lastmodifytime'][0] + " and " + df['plt_createtime'][0]

In [25]:
def test2(df):
#     print(df)
    return len(df)

In [26]:
def test3(df):
#     print(df)
    return len(df)

### 数据图

#### 所有数据

In [27]:
MIN_64BIT_SIGNED_INTERGER = -2**63 
MAX_64BIT_SIGNED_INTERGER = 2**63 - 1
def change2float(x):
    if re.match("^\d+$", str(x)) is not None:
        if MIN_64BIT_SIGNED_INTERGER < int(x) < MAX_64BIT_SIGNED_INTERGER:
            return int(x)
        else:
            return float(x)
    elif re.match("^\d+?(\.\d+?)?$", str(x)) is not None:
        return float(x)
    else:
        return x
def test_change2float(s):
    print(change2float(s), "\t", type(change2float(s)))

test_change2float("1")
test_change2float("1.03")
test_change2float("1.0a")
test_change2float("1000000")
test_change2float("12345679900000000000")

1 	 <class 'int'>
1.03 	 <class 'float'>
1.0a 	 <class 'str'>
1000000 	 <class 'int'>
1.23456799e+19 	 <class 'float'>


In [28]:
graph.nodes.match("服务人员使用车", value='1070105279').first()

In [29]:
# if option_deleteAllFlag == 1:
#     graph.delete_all()

# table_name = 'plt_intesrv_tydevice'
for table_name in tables_name:
    print(table_name)
    # 取出属于本表的几个小表，包括实体数据表、概念表、关系表
    df_table = tables[table_name]
    df_concept_one_table = df_concept[df_concept['table'] == table_name]
    df_relation_one_table = df_relation[df_relation['table'] == table_name]
#     print("debug:df_concept_one_table=%s"%df_concept_one_table)
#     print("debug:df_relation_one_table=%s"%df_relation_one_table)

    concept_ch_list_one_table = list(df_concept_one_table['concept_ch'])
    row_count = 0
    # 对于数据表的每行进行处理
    for i_row in tqdm(range(0,df_table.shape[0])): 
#         print(i_row)
#         if row_count > 3:
#             break
        row_count += 1
        entity_node = {}
        # 对于概念表中的每个概念进行处理
        for i_entity in list(df_concept_one_table.index):
            key = df_concept_one_table['concept_ch'][i_entity] # eg:"客户","挖掘机"
            if df_concept_one_table['type'][i_entity] == '直接':
#                 print(df_concept_one_table['col'][i_entity])
                value = df_table[ df_concept_one_table['col'][i_entity] ][i_row] # eg:"张三","PC200"
                if value != 'nan': # 如果该节点的值非空
#                     value = str(value).strip()
                    nodeBefore = graph.nodes.match(key, value=change2float(value)).first()
                    if nodeBefore is None:
                        aNode = Node(key)
                        aNode['value'] = change2float(value)
                        aNode['type'] = key
                        aNode.add_label("实例节点") # Neo4j的官方label，支持高效索引

                        # 为该节点增加对应属性
                        assert len(df_concept_one_table['properties_col'][i_entity]) == len(df_concept_one_table['properties_col_ch'][i_entity]) \
                            , "properties_col=%s, properties_col_ch=%s. They have different length."%(str(df_concept_one_table['properties_col'][i_entity]), str(df_concept_one_table['properties_col_ch'][i_entity]))
                        prop_num = len(df_concept_one_table['properties_col'][i_entity])
                        for i_prop in range(0,prop_num):
                            prop_key = df_concept_one_table['properties_col'][i_entity][i_prop]  # eg: prop_key = 'plt_devicebrand'
                            prop_key_ch = df_concept_one_table['properties_col_ch'][i_entity][i_prop] # eg: prop_key_ch = '品牌'
                            prop_value = df_table[prop_key][i_row]
                            if prop_value != 'nan': 
                                aNode[prop_key_ch] = change2float(prop_value)
                            else: # 属性值如果为空的话，会置为空字符串
                                aNode[prop_key_ch] = ''

                        # 为该节点增加display_name属性
    #                     display = "(" + str(key) + ")"
#                         display = str(key) + ":"
                        display = ""
                        display_col = df_concept_one_table['display_name'][i_entity]
                        if display_col == 'unique_id':
                            display += value 
                        elif display_col == "nan": # 如果那一列为空的话，display就只显示概念名，不显示具体值
#                             display = display[:-2]
                            pass
                        else:
                            # 根据中文列名找到数据库列名
                            prop_num = len(df_concept_one_table['properties_col'][i_entity])
                            for i_prop in range(0,prop_num):
                                prop_key = df_concept_one_table['properties_col'][i_entity][i_prop]  # eg: prop_key = 'plt_devicebrand'
                                prop_key_ch = df_concept_one_table['properties_col_ch'][i_entity][i_prop] # eg: prop_key_ch = '品牌'
                                if prop_key_ch == display_col:
                                    break
                            assert i_prop != prop_num - 1 or (i_prop == prop_num - 1 and prop_key_ch == display_col) \
                                , "display_col=%s, properties_col_ch=%s. display_col doesn't exist in properties_col_ch."%(display_col, str(df_concept_one_table['properties_col_ch'][i_entity]))
                            display += str(df_table[prop_key][i_row])
#                         print("debug:display=", display)
                        aNode['display_name'] = display
                        if option_label != "":
                            aNode.add_label(option_label)
                            aNode['domain_name'] = option_label
    #                     print(aNode)
                        graph.create(aNode)
                    else:
                        aNode = nodeBefore
                    # 因为同一概念可能对应多列，需要将entity_node的key修改为数据库表中的列名
                    key = df_concept_one_table['col'][i_entity]
                    entity_node[key] = aNode
            # 生成合成概念实体
            elif df_concept_one_table['type'][i_entity] == '合成概念':
#                 print(df_concept_one_table['concept_ch'][i_entity])
#                 print(df_concept_one_table['compound_table'][i_entity])
                if str(df_concept_one_table['compound_table'][i_entity]) == 'nan': # 非空的话只需要把当前所在行传入函数
                    indexs = [] 
                    indexs.append(i_row)
                    # use list as index to return DataFrame instead of Series though the list has only one element
                    df_compound_need = df_table.loc[indexs,:].reset_index(drop=True) 
                else: # 处理联合多表的情况
                    compound = df_concept_one_table['compound_table'][i_entity][0]
                    assert table_name == compound['main_table'] # 第一个跳转表的主表应该得是本表
                    main_key_value = df_table[compound['main_col']][i_row]
                    df_table_new = tables[compound['foreign_table']]
                    foreign_col = compound['foreign_col']
                    df_compound_need = df_table_new.loc[df_table_new[foreign_col] == main_key_value]
                    # TODO:处理联合多表的情况
                value = eval(df_concept_one_table['code'][i_entity] + "(df_compound_need)")
                aNode = Node(key)
                aNode.add_label("实例节点")
                aNode['value'] = change2float(value)
                aNode['type'] = key
#                 aNode['display_name'] = str(key) + ":" + str(value) 
                aNode['display_name'] = str(value) 
                if option_label != "":
                    aNode.add_label(option_label)
                    aNode['domain_name'] = option_label
                graph.create(aNode)
                entity_node[key] = aNode
#         print("debug:entity_node=%s"%entity_node)
#         print("debug:df_relation_one_table=%s"%df_relation_one_table)
        # 建立本行数据中的节点间的关系
        for i_relation in list(df_relation_one_table.index):
    #         try: 
            # 建立sub主语实体，有可能已经在entity_node中，也有可能是外键
            entity_ch = df_relation_one_table['sub'][i_relation]
            entity_col_name = ''
            if df_relation_one_table['sub_col'][i_relation] != "nan":
#                 print('euqal')
                entity_col_name = df_relation_one_table['sub_col'][i_relation]
                if entity_col_name in entity_node.keys(): # 本行数据确实有该节点
                    entity = entity_node[entity_col_name]
                else:
                    key = entity_ch
                    value = df_table[entity_col_name][i_row]
                    if value != value: # 虽然该键是外键，但是该键依然可能是空值(玄学数据)
                        continue
#                     value = str(value).strip()
                    nodeBefore = graph.nodes.match(key, value=change2float(value)).first()
        #             assert nodeBefore is not None
                    if nodeBefore is None: # 这是测试数据的锅，否则的话永远不应该为None的
                        continue
                    entity = nodeBefore
#                 print("debug:entity_col_name=%s"%entity_col_name)
            else:
#                 print("debug:entity_ch=%s"%entity_ch)
#                 print(df_concept_one_table)
#                 print(df_concept)
#                 print("debug:df_concept_one_table.loc[(df_concept['concept_ch'] == entity_ch), 'col']=")
#                 print(df_concept_one_table.loc[(df_concept['concept_ch'] == entity_ch), 'col'])
#                 entity_col_name = df_concept_one_table.loc[(df_concept_one_table['concept_ch'] == entity_ch), 'col'].item()
#             print(entity_col_name)
#             print("df_relation_one_table['sub_col'][i_relation]=%s"%df_relation_one_table['sub_col'][i_relation])
                if entity_ch in df_concept_one_table['concept_ch'].tolist(): # 本行数据确实有该节点
                    entity_col_name = df_concept_one_table.loc[(df_concept_one_table['concept_ch'] == entity_ch), 'col'].item()
                    if entity_col_name in entity_node.keys(): # 本行数据确实有该节点
                        entity = entity_node[entity_col_name]
                    else:
                        continue
                elif entity_ch not in concept_ch_list_one_table: # 本行数据无该节点，则为外键
                    # entity_ch(eg:客户)去模式表里找该概念的数据库主键来源，根据主键和当前表table_name来找到该概念在当前表的外键列名，然后根据该外键列名找到外键具体值，去neo4j里查询该节点，
    #                 print(entity_ch)
                    if df_relation_one_table['sub_col'][i_relation] != "nan":
                        key = entity_ch
                        value = df_table[entity_col_name][i_row]
                    else:
                        primary_key_table = df_concept.loc[df_concept['concept_ch'] == entity_ch, 'table'].item()
                        primary_key_col = df_concept.loc[df_concept['concept_ch'] == entity_ch, 'col'].item()
#                         print(df_relation_one_table.loc[i_relation])
#                         print("debug:primary_key_table=%s"%primary_key_table)
#                         print("debug:primary_key_col=%s"%primary_key_col)
#                         print("debug:entity_ch=%s"%entity_ch)
#                         print(df_foreign['primary_table'] == primary_key_table)
#                         print((df_foreign['primary_key'] == primary_key_col))
#                         print((df_foreign['foreign_table'] == table_name))
#                         print(df_foreign.loc[(df_foreign['primary_table'] == primary_key_table)
#                                           & (df_foreign['primary_key'] == primary_key_col)
#                                           & (df_foreign['foreign_table'] == table_name), 'foreign_key'])
                        foreign_key = df_foreign.loc[(df_foreign['primary_table'] == primary_key_table)
                                          & (df_foreign['primary_key'] == primary_key_col)
                                          & (df_foreign['foreign_table'] == table_name), 'foreign_key'].item()
        #                 print(foreign_key)
                        key = entity_ch
                        value = df_table[foreign_key][i_row]
                    if value != value: # 虽然该键是外键，但是该键依然可能是空值(玄学数据)
                        continue
#                     value = str(value).strip()
                    nodeBefore = graph.nodes.match(key, value=change2float(value)).first()
        #             assert nodeBefore is not None
                    if nodeBefore is None: # 这是测试数据的锅，否则的话永远不应该为None的
                        continue
                    entity = nodeBefore
        #             print(type(primary_key_table)) 
        #             primary_key_table, primary_key_col
        #             continue
                else: # 本行数据的该节点value为空，所以没有该节点
                    continue
            sub = entity

            # 建立obj宾语实体，这二三十航代码都是抄的上面的，只要把首末行的sub改成obj就可以了
            entity_ch = df_relation_one_table['obj'][i_relation]
            entity_col_name = ''
            if df_relation_one_table['obj_col'][i_relation] != "nan":
#                 print('euqal')
                entity_col_name = df_relation_one_table['obj_col'][i_relation]
                if entity_col_name in entity_node.keys(): # 本行数据确实有该节点
                    entity = entity_node[entity_col_name]
                else: # 本行数据无该节点，则为外键
                    key = entity_ch
                    value = df_table[entity_col_name][i_row]
                    if value != value: # 虽然该键是外键，但是该键依然可能是空值(玄学数据)
                        continue
#                     value = str(value).strip()
                    nodeBefore = graph.nodes.match(key, value=change2float(value)).first()
        #             assert nodeBefore is not None
                    if nodeBefore is None: # 这是测试数据的锅，否则的话永远不应该为None的
                        continue
                    entity = nodeBefore
#                 print("debug:entity_col_name=%s"%entity_col_name)
            else:
#                 print("debug:entity_ch=%s"%entity_ch)
#                 print(df_concept_one_table)
#                 print(df_concept)
#                 print("debug:df_concept_one_table.loc[(df_concept['concept_ch'] == entity_ch), 'col']=")
#                 print(df_concept_one_table.loc[(df_concept['concept_ch'] == entity_ch), 'col'])
#                 entity_col_name = df_concept_one_table.loc[(df_concept_one_table['concept_ch'] == entity_ch), 'col'].item()
#             print(entity_col_name)
#             print("df_relation_one_table['obj_col'][i_relation]=%s"%df_relation_one_table['obj_col'][i_relation])
                if entity_ch in df_concept_one_table['concept_ch'].tolist(): # 本行数据确实有该节点
                    entity_col_name = df_concept_one_table.loc[(df_concept_one_table['concept_ch'] == entity_ch), 'col'].item()
                    if entity_col_name in entity_node.keys(): # 本行数据确实有该节点
                        entity = entity_node[entity_col_name]
                    else:
                        continue
                elif entity_ch not in concept_ch_list_one_table: # 本行数据无该节点，则为外键
                    # entity_ch(eg:客户)去模式表里找该概念的数据库主键来源，根据主键和当前表table_name来找到该概念在当前表的外键列名，然后根据该外键列名找到外键具体值，去neo4j里查询该节点，
    #                 print(entity_ch)
                    if df_relation_one_table['obj_col'][i_relation] != "nan":
                        key = entity_ch
                        value = df_table[entity_col_name][i_row]
                    else:
                        primary_key_table = df_concept.loc[df_concept['concept_ch'] == entity_ch, 'table'].item()
                        primary_key_col = df_concept.loc[df_concept['concept_ch'] == entity_ch, 'col'].item()
    #                     print(df_relation_one_table.loc[i_relation])
    #                     print("debug:primary_key_table=%s"%primary_key_table)
    #                     print("debug:primary_key_col=%s"%primary_key_col)
    #                     print("debug:entity_ch=%s"%entity_ch)
    #                     print(df_foreign['primary_table'] == primary_key_table)
    #                     print((df_foreign['primary_key'] == primary_key_col))
    #                     print((df_foreign['foreign_table'] == table_name))
    #                     print(df_foreign.loc[(df_foreign['primary_table'] == primary_key_table)
    #                                       & (df_foreign['primary_key'] == primary_key_col)
    #                                       & (df_foreign['foreign_table'] == table_name), 'foreign_key'])
                        foreign_key = df_foreign.loc[(df_foreign['primary_table'] == primary_key_table)
                                          & (df_foreign['primary_key'] == primary_key_col)
                                          & (df_foreign['foreign_table'] == table_name), 'foreign_key'].item()
        #                 print(foreign_key)
                        key = entity_ch
                        value = df_table[foreign_key][i_row]
                    if value != value: # 虽然该键是外键，但是该键依然可能是空值(玄学数据)
                        continue
#                     value = str(value).strip()
                    nodeBefore = graph.nodes.match(key, value=change2float(value)).first()
        #             assert nodeBefore is not None
                    if nodeBefore is None: # 这是测试数据的锅，否则的话永远不应该为None的
                        continue
                    entity = nodeBefore
        #             print(type(primary_key_table)) 
        #             primary_key_table, primary_key_col
        #             continue
                else: # 本行数据的该节点value为空，所以没有该节点
                    continue
            obj = entity

            # 建立主宾实体间的关系
            predicate = df_relation_one_table['predicate'][i_relation]
#             print("debug:sub=",sub)
#             print("debug:obj=",obj)
            aRelation = Relationship(sub, predicate, obj)
            aRelation['type'] = predicate
            aRelation['id_relation'] = df_relation_one_table['id'][i_relation]
            # 为关系增加属性
            assert len(df_relation_one_table['prop_ch'][i_relation]) == len(df_relation_one_table['prop_col'][i_relation]) \
                , "prop_ch=%s, prop_col=%s. They have different length."%(str(df_relation_one_table['prop_ch'][i_relation]), str(df_relation_one_table['prop_col'][i_relation]))
            prop_num = len(df_relation_one_table['prop_ch'][i_relation])
            for i_prop in range(0,prop_num):
                prop_key = df_relation_one_table['prop_col'][i_relation][i_prop]
                prop_key_ch = df_relation_one_table['prop_ch'][i_relation][i_prop]
                prop_value = df_table[prop_key][i_row]
                if prop_value != 'nan':
                    aRelation[prop_key_ch] = change2float(prop_value)
                else:
                    aRelation[prop_key_ch] = ''
#             print(aRelation)
            graph.create(aRelation)
    #         except KeyError: # 因为某些节点可能因为值为空所以没有该节点，所以在entity_node字典中没有该节点
    #             pass
    #         print(entity_name)

devicemodel_entity


100%|█████████████████████████████████████████████████████████████████████████████████| 70/70 [00:00<00:00, 948.47it/s]


srvpersontrainrecord_falutpart


100%|█████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 601.71it/s]


srvpersonlevel


...

100%|████████████████████████████████████████████████████████████████████████████████| 176/176 [00:12<00:00, 14.26it/s]


devicemodel_relation


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:06<00:00, 11.25it/s]


srvpersoncar


100%|██████████████████████████████████████████████████████████████████████████████████| 82/82 [00:02<00:00, 26.41it/s]
